In [1]:

from datasets import load_dataset
from tqdm import tqdm

dataset = load_dataset("roneneldan/TinyStories")

Found cached dataset parquet (C:/Users/scott/.cache/huggingface/datasets/roneneldan___parquet/roneneldan--TinyStories-6ac769f186d7da53/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# print(dataset)
# print(dataset.keys())
train_dset = dataset['train']
print(train_dset[0])
# print(len(train_dset))
print(train_dset[0]['text'])

{'text': 'One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.\n\nLily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."\n\nTogether, they shared the needle and sewed the button on Lily\'s shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.'}
One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.

Lily went to her mom and said, "Mom, I found this needle. Can you sh

In [ ]:
def integer_encode(texts):
    word_to_index = {}
    index_to_word = {}
    encoded_texts = []

    # Assign a unique integer to each word
    current_index = 1  # Start indexing from 1, leaving 0 for padding if needed
    for i in tqdm(range(len(texts))):
        text = train_dset[i]['text']
        encoded_text = []
        for word in text.split():
            if word not in word_to_index:
                word_to_index[word] = current_index
                index_to_word[current_index] = word
                current_index += 1
            encoded_text.append(word_to_index[word])
        encoded_texts.append(encoded_text)

    return word_to_index, index_to_word, encoded_texts

In [ ]:
w_to_i, i_to_w, encoded_train = integer_encode(train_dset)

100%|██████████| 2119719/2119719 [04:31<00:00, 7798.94it/s]


In [ ]:
print(encoded_train[0])
print(len(w_to_i))

[1, 2, 3, 4, 5, 6, 7, 8, 3, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 15, 21, 15, 16, 22, 7, 23, 18, 24, 25, 9, 20, 11, 26, 27, 28, 29, 30, 3, 31, 32, 11, 33, 7, 34, 18, 11, 35, 36, 37, 38, 39, 8, 40, 41, 42, 43, 24, 15, 20, 44, 36, 30, 45, 46, 47, 35, 48, 36, 37, 49, 50, 51, 52, 24, 25, 9, 36, 53, 54, 55, 56, 57, 58, 25, 9, 36, 59, 25, 31, 32, 60, 33, 61, 16, 62, 17, 63, 64, 21, 57, 65, 66, 36, 67, 68, 69, 70, 57, 71, 7, 72, 11, 35, 63, 66, 25, 9, 36, 73, 11, 33, 74, 75, 76, 77, 21, 57, 78, 58, 36, 79, 80]
265840


In [ ]:
# torch and torchvision imports
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from torch.optim import lr_scheduler
import numpy as np
import matplotlib.pyplot as plt
import re
import math
import os
from tempfile import TemporaryDirectory
from typing import Tuple
from torch import nn, Tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
# device = torch.device('cuda')

In [ ]:
tran_dataset_input = []
tran_dataset_label = []

for i in tqdm(range(len(encoded_train))):
  try:
    tran_dataset_input.append(np.array(encoded_train[i][:100]))
    tran_dataset_label.append(np.array(encoded_train[i][1:101]))
  except:
    pass

100%|██████████| 2119719/2119719 [00:45<00:00, 46994.50it/s]


In [ ]:

print(len(tran_dataset_input[0]))
print(len(tran_dataset_input))
print(len(tran_dataset_label[0]))

100
2119719
100


In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, max_len: int = 5000):
        super().__init__()

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + (self.pe[:x.size(0)]).squeeze()
        return x

class TransformerModel(nn.Module):

    def __init__(self, input_feature_dim, d_model, nhead, nlayers, embedding_size):
        super(TransformerModel, self).__init__()

        self.embedding = nn.Embedding(embedding_size, d_model)
        encoder_layers = TransformerEncoderLayer(d_model = d_model, nhead = nhead)
        self.transformer = TransformerEncoder(encoder_layers, nlayers)
        self.linear = nn.Linear(d_model, input_feature_dim)
        self.pos_encoder = PositionalEncoding(d_model)

    def generate_mask(self, sz):
      mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
      mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
      return mask

    def forward(self, x):
        """
        Arguments:
            src: Tensor, shape ``[seq_len, batch_size]``
            src_mask: Tensor, shape ``[seq_len, seq_len]``

        Returns:
            output Tensor of shape ``[seq_len, batch_size, ntoken]``
        """
        x = self.embedding(x)
        x = self.pos_encoder(x)
        mask = self.generate_mask(len(x)).to(device)
        x = self.transformer(x, mask)

        x = self.linear(x)

        return x

In [ ]:
embed_len = 265840
vocab_size = 265840
heads = 8
layers = 6
hidden_size = 3 * vocab_size
input_feature_dim = 100
tran = TransformerModel(input_feature_dim, hidden_size-(hidden_size % heads), nhead = heads, nlayers = layers,  embedding_size = embed_len).to(device)
# print(hidden_size-(hidden_size % heads))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(tran.parameters(), lr = 0.0001)

In [ ]:
tran_train_l = []
tran_train_e = []
tran_val_l = []
tran_val_e = []

num_epochs = 1
i = 0
for epoch in range(num_epochs):

    for sequence in tran_dataset_input:
        print(i)
        tran.zero_grad()

        sequence = torch.from_numpy(sequence.T).long()
        sequence = sequence.to(device)
        # print(sequence.shape)
        output = tran(sequence)

        current_labels = torch.from_numpy(tran_dataset_label[i])
        current_labels = current_labels.to(device)
        # try:
        loss = criterion(output,current_labels)
        # except:
        #   break

        optimizer.zero_grad()
        loss.backward()

        if (i % 1000) != 0:
            optimizer.step()

        tran_train_l.append(loss.item())
        predicted_labels = torch.argmax(output,dim=1)
        train_error = 1.0 - (current_labels == predicted_labels).sum()/(len(sequence)-1)
        tran_train_e.append(train_error.item())
        i += 1

        # if (i % 1000) == 0:
        #     # Validation
        #     val_sequence = tran_dataset_input[i+1]
        #     val_sequence = torch.from_numpy(val_sequence).long().to(device)
        #     val_output = tran(val_sequence)
        #     val_labels = torch.from_numpy(tran_dataset_label[i+1]).to(device)
        #     val_loss = criterion(val_output, val_labels)
        #     tran_val_l.append(val_loss.item())
        #     future_predicted_labels = torch.argmax(val_output, dim=1)
        #     validation_error = 1.0 - (val_labels == future_predicted_labels).sum() / (len(val_sequence) - 1)
        #     print(f'training_error: {train_error}')
        #     print(f'validation_error: {validation_error}')
        #     tran_val_e.append(validation_error.item())
        #     print(f'Currently at {(i / len(tran_dataset_input)) * 100}% complete')